In [31]:
from astropy.table import Table, MaskedColumn
from astropy.time import Time
import astropy.io.ascii
import numpy as np

In [54]:
%run -b -i "../python/calculator.py"

In [33]:
traw = Table.read('../data/asset1_transactions.csv', format='ascii.csv', converters={
                  'quantity': [astropy.io.ascii.convert_numpy(np.float64)],
                  'unit price': [astropy.io.ascii.convert_numpy(np.float64)],
                  'total price': [astropy.io.ascii.convert_numpy(np.float64)]})

In [34]:
vraw = Table.read('../data/asset1_values.csv', format='ascii.csv', converters={
                  'price': [astropy.io.ascii.convert_numpy(np.float64)],})
vraw['date'] = Time(vraw['date'])

In [35]:
prepareRawTransactions(traw)

dateS,asset,type,quantity,unit price,total price,dateE
object,str2,str4,float64,float64,float64,object
2019-01-01 00:00:00.000,A1,buy,5.0,2.0,10.0,--
2020-01-01 00:00:00.000,A1,buy,10.0,3.0,30.0,--
2021-01-01 00:00:00.000,A1,sell,1.0,5.0,5.0,--


In [36]:
packets = packetizeTransactions(traw, 'A1_')

Sale 2021-01-01:1.0@5.0 -> 
Buys 
2019-01-01:5.0@2.0
2020-01-01:10.0@3.0
Using buy 0
Remaining from buy : 4.0


In [37]:
packets

dateS,asset,quantity,priceS,dateE,priceE,packetId
object,str2,float64,float64,object,float64,bytes20
2019-01-01 00:00:00.000,A1,1.0,2.0,2021-01-01 00:00:00.000,5.0,A1_001
2019-01-01 00:00:00.000,A1,4.0,2.0,--,--,A1_003
2020-01-01 00:00:00.000,A1,10.0,3.0,--,--,A1_002


In [38]:
vraw

date,price
object,float64
2019-01-01 00:00:00.000,2.0
2020-01-01 00:00:00.000,3.0
2021-01-01 00:00:00.000,5.0
2022-01-01 00:00:00.000,6.0
2023-01-01 00:00:00.000,7.0


In [39]:
packetGainsT, totalGainsT = calculateOneYearOneAssetTaxes(packets, 'A1', 2020, vraw, firstMMTaxYear=True)

In [40]:
calculateOneYearOneAssetTaxes(packets, 'A1', 2021, vraw, packetGainsT=packetGainsT, totalGainsT=totalGainsT, firstMMTaxYear=False)

(<Table length=6>
 tyear assetId packetId oBasisS MMBasisS  regInc ltcGain oBasisE MMBasisE urevInc
 int64 bytes20 bytes20  float64 float64  float64 float64 float64 float64  float64
 ----- ------- -------- ------- -------- ------- ------- ------- -------- -------
  2020      A1   A1_001     2.0      3.0     2.0     0.0     4.0      5.0     2.0
  2020      A1   A1_003     2.0      3.0     8.0     0.0     4.0      5.0    10.0
  2020      A1   A1_002     3.0      3.0    20.0     0.0     5.0      5.0    30.0
  2021      A1   A1_001     4.0      5.0     0.0     1.0     5.0      5.0    30.0
  2021      A1   A1_003     4.0      5.0     4.0     0.0     5.0      6.0    34.0
  2021      A1   A1_002     5.0      5.0    10.0     0.0     6.0      6.0    44.0,
 <Table length=2>
 tyear assetId urevIncS urevIncE  regInc ltcGain
 int64 bytes20 float64  float64  float64 float64
 ----- ------- -------- -------- ------- -------
  2020      A1      0.0     30.0    30.0     0.0
  2021      A1     30.0     4

In [41]:
calculateOneYearOneAssetTaxes(packets, 'A1', 2022, vraw, packetGainsT=packetGainsT, totalGainsT=totalGainsT, firstMMTaxYear=False)

(<Table length=8>
 tyear assetId packetId oBasisS MMBasisS  regInc ltcGain oBasisE MMBasisE urevInc
 int64 bytes20 bytes20  float64 float64  float64 float64 float64 float64  float64
 ----- ------- -------- ------- -------- ------- ------- ------- -------- -------
  2020      A1   A1_001     2.0      3.0     2.0     0.0     4.0      5.0     2.0
  2020      A1   A1_003     2.0      3.0     8.0     0.0     4.0      5.0    10.0
  2020      A1   A1_002     3.0      3.0    20.0     0.0     5.0      5.0    30.0
  2021      A1   A1_001     4.0      5.0     0.0     1.0     5.0      5.0    30.0
  2021      A1   A1_003     4.0      5.0     4.0     0.0     5.0      6.0    34.0
  2021      A1   A1_002     5.0      5.0    10.0     0.0     6.0      6.0    44.0
  2022      A1   A1_003     5.0      6.0     4.0     0.0     6.0      7.0    48.0
  2022      A1   A1_002     6.0      6.0    10.0     0.0     7.0      7.0    58.0,
 <Table length=3>
 tyear assetId urevIncS urevIncE  regInc ltcGain
 int64 bytes

In [55]:
q = getValue(Time("2021-12-01"),vraw, 'date', 'price')

In [56]:
q

5.9150684931506845

In [49]:
q.value

5.9150684931506845

In [50]:
q

<Quantity 5.91506849>

In [53]:
q.value

5.9150684931506845